# Prompt Experiments

In this notebook we will experiment with prompts and the OpenAI models. We will try introducing different concepts and prompts using the very capable GPT-4 family of models. 


### Python Imports


In [1]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('..\\code')


import os
from dotenv import load_dotenv
load_dotenv()

from IPython.display import display, Markdown, HTML
from PIL import Image
from doc_utils import *


def show_img(img_path, width = None):
    if width is not None:
        display(HTML(f'<img src="{img_path}" width={width}>'))
    else:
        display(Image.open(img_path))


### Make sure we have the OpenAI Models information

We will need the GPT-4-Turbo and GPT-4-Vision models for this notebook.

When running the below cell, the values should reflect the OpenAI reource you have created in the `.env` file.

In [3]:
model_info = {
        'AZURE_OPENAI_RESOURCE': os.environ.get('AZURE_OPENAI_RESOURCE'),
        'AZURE_OPENAI_KEY': os.environ.get('AZURE_OPENAI_KEY'),
        'AZURE_OPENAI_MODEL_VISION': os.environ.get('AZURE_OPENAI_MODEL_VISION'),
        'AZURE_OPENAI_MODEL': os.environ.get('AZURE_OPENAI_MODEL'),
}

### Generate Sample Data

Generate the sample images that will be used in this notebook

In [4]:
import fitz  # PyMuPDF

# Create a directory to store the outputs
work_dir = "sample_data/pdf_outputs"
os.makedirs(work_dir, exist_ok=True)

# Load a sample PDF document

def read_pdf(pdf_doc):
    doc = fitz.open(pdf_doc)
    print(f"PDF File {os.path.basename(pdf_doc)} has {len(doc)} pages.")
    return doc

def nb_extract_pages_as_png_files(doc):
    png_files = []
    for page in doc:
        page_num = page.number
        img_path = f"{work_dir}/page_{page_num}.png"
        page_pix = page.get_pixmap(dpi=300)
        page_pix.save(img_path)
        print(f"Page {page_num} saved as {img_path}")
        png_files.append(img_path)
    
    return png_files


pdf_doc = "sample_data/1_London_Brochure.pdf"
doc = read_pdf(pdf_doc)
png_files = nb_extract_pages_as_png_files(doc)  


PDF File 1_London_Brochure.pdf has 2 pages.
Page 0 saved as sample_data/pdf_outputs/page_0.png
Page 1 saved as sample_data/pdf_outputs/page_1.png


## Visual Element Detection

The first function we will experiment with is Detection. Sometimes, we need to detect whether a page has an image or embedded inside it, or a table, using a Vision model.

#### Table Detection

In the below cell, we are trying to detect tables in the png files generated from the PDF document. The code below will display as an image the pages where tables were found.

In [20]:
detect_num_of_tables_prompt = """
You are an assistant working on a document processing task that involves detecting and counting the number of data tables in am image file using a vision model. Given an image, your task is determine the number of data tables present. 

Output:
Return a single integer representing the number of data tables detected in the page. Do **NOT** generate any other text or explanation, just the number of tables. We are **NOT** looking for the word 'table' in the text, we are looking for the number of data tables in the image.

"""
for png in png_files:
    result, description = call_gpt4v(png, gpt4v_prompt = detect_num_of_tables_prompt, temperature = 0.2, model_info=model_info)
    print(f"Status: {description}")
    print(f"Result: {result} tables detected in the PDF page.")

    if int(result) > 0:
        show_img(png, width=400)



16.03.2024_16.07.52 :: Start of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_0.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
endpoint https://oai-tst-sweden.openai.azure.com/openai/deployments/gpt4v/extensions/chat/completions?api-version=2023-12-01-preview

16.03.2024_16.07.58 :: End of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_0.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
Status: Image was successfully explained, with Status Code: 200
Result: 0 tables detected in the PDF page.

16.03.2024_16.07.58 :: Start of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_1.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
endpoint https://oai-tst-sweden.openai.azure.com/openai/deployments/gpt4v/extensions/chat/completions?api-version=2023-12-01-preview

16.03.2024_16.08.04 :: End of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_1.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
Status:

#### Image Detection

In the below cell, we are trying to detect images in the png files generated from the PDF document. The code below will display as an image the pages where images were found.

In [21]:
detect_num_of_diagrams_prompt = """
You are an assistant working on a document processing task that involves detecting and counting the number of visual assets in a document page using a vision model. Given a screenshot of a documenat page, your task is determine the number of visual assets present. Please ignore any standard non-visual assets such as text, headers, footers, page numbers, tables, etc.

What is meant by visual assets: infographics, maps, flowcharts, timelines, tables, illustrations, icons, heatmaps, scatter plots, pie charts, bar graphs, line graphs, histograms, Venn diagrams, organizational charts, mind maps, Gantt charts, tree diagrams, pictograms, schematics, blueprints, 3D models, storyboards, wireframes, dashboards, comic strips, story maps, process diagrams, network diagrams, bubble charts, area charts, radar charts, waterfall charts, funnel charts, sunburst charts, sankey diagrams, choropleth maps, isometric drawings, exploded views, photomontages, collages, mood boards, concept maps, fishbone diagrams, decision trees, Pareto charts, control charts, spider charts, images, diagrams, logos, charts or graphs.

Output:
Return a single integer representing the number of visual assets detected in the page. Do **NOT** generate any other text or explanation, just the count of . 

"""

for png in png_files:
    result, description = call_gpt4v(png, gpt4v_prompt = detect_num_of_diagrams_prompt, temperature = 0.2, model_info=model_info)
    print(f"Status: {description}")
    print(f"Result: {result} images detected in the PDF page.")

    if int(result) > 0:
        show_img(png, width=400)



16.03.2024_16.08.44 :: Start of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_0.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
endpoint https://oai-tst-sweden.openai.azure.com/openai/deployments/gpt4v/extensions/chat/completions?api-version=2023-12-01-preview

16.03.2024_16.08.50 :: End of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_0.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
Status: Image was successfully explained, with Status Code: 200
Result: 2 images detected in the PDF page.



16.03.2024_16.08.50 :: Start of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_1.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
endpoint https://oai-tst-sweden.openai.azure.com/openai/deployments/gpt4v/extensions/chat/completions?api-version=2023-12-01-preview

16.03.2024_16.08.55 :: End of GPT4V Call to process file(s) ['sample_data/pdf_outputs/page_1.png'] with model: https://oai-tst-sweden.openai.azure.com/ 
Status: Image was successfully explained, with Status Code: 200
Result: 0 images detected in the PDF page.


## Analyze Code

Read in the doc_utils.py library, and generate the function tree for the `ingest_docs_directory` function. 

In [3]:
prompt = """

{code}

In the library above, detect the important functions in the '{func_name}' tree. Start with the '{func_name}' function, and then list the essential functions called by that '{func_name}'. Please ignore small functions that are 3 lines of code or less. Focus only on the custom functions defined in the code using the keyword 'def', and ignore the imported functions or system functions like fitz.open and glob.glob. Then please do the following:

    1. Output the list in bullet-point format
    2. Show their relationship by generating Mermaid code that represents these functions.
    3. Make sure the relationships are correct. The arrow should point from the calling function to the called function.

"""

p = prompt.format(code = read_asset_file("../code/doc_utils.py")[0], func_name = 'ingest_docs_directory')

output = ask_LLM(p, model_info=model_info)
print(output)



- `ingest_docs_directory`
  - `ingest_doc`
    - `process_pdf`
      - `extract_high_res_page_images`
      - `extract_text`
      - `harvest_code`
      - `extract_images`
      - `post_process_images`
      - `extract_tables`
      - `post_process_tables`
    - `commit_assets_to_vector_index`
      - `add_asset_to_vec_store`
        - `create_metadata`
        - `get_embeddings`
        - `get_solution_path_components`
        - `generate_uuid_from_string`
    - `save_docx_as_pdf`
    - `save_pptx_as_pdf`

```mermaid
graph TD;
    ingest_docs_directory --> ingest_doc;
    ingest_doc --> process_pdf;
    process_pdf --> extract_high_res_page_images;
    process_pdf --> extract_text;
    process_pdf --> harvest_code;
    process_pdf --> extract_images;
    process_pdf --> post_process_images;
    process_pdf --> extract_tables;
    process_pdf --> post_process_tables;
    ingest_doc --> commit_assets_to_vector_index;
    commit_assets_to_vector_index --> add_asset_to_vec_store;
    add

### Render the Mermaid code

To render the above generated Mermaid Code, please copy the above Mermaid script block, visit [mermaid.live](https://mermaid.live) in your browser, and paste the copied script in your browser. 

The image should be rendered immediately.

![Mermaid Representation](../images/ingestion_tree.png)

Now do the `search` function.

In [4]:
prompt = """

{code}

In the library above, detect the important functions in the '{func_name}' tree. Start with the '{func_name}' function, and then list the essential functions called by that '{func_name}'. Please ignore small functions that are 3 lines of code or less. Focus only on the custom functions defined in the code using the keyword 'def', and ignore the imported functions or system functions like fitz.open and glob.glob. Then please do the following:

    1. Output the list in bullet-point format
    2. Show their relationship by generating Mermaid code that represents these functions.
    3. Make sure the relationships are correct. The arrow should point from the calling function to the called function.

"""

p = prompt.format(code = read_asset_file("../code/doc_utils.py")[0], func_name = 'search')

output = ask_LLM(p, model_info=model_info)
print(output)


- `search`
  - `aggregate_ai_search`
    - `get_query_entities`
    - `call_ai_search`
  - `generate_search_assets`
  - `apply_computation_support`
    - `try_code_interpreter_for_tables_using_taskweaver`
    - `try_code_interpreter_for_tables_using_python_exec`
    - `try_code_interpreter_for_tables_using_assistants_api`
  - `clean_up_text`
  - `generate_search_assets`
  - `process_assistants_api_response`

```mermaid
graph TD
    search --> aggregate_ai_search
    aggregate_ai_search --> get_query_entities
    aggregate_ai_search --> call_ai_search
    search --> generate_search_assets
    search --> apply_computation_support
    apply_computation_support --> try_code_interpreter_for_tables_using_taskweaver
    apply_computation_support --> try_code_interpreter_for_tables_using_python_exec
    apply_computation_support --> try_code_interpreter_for_tables_using_assistants_api
    search --> clean_up_text
    search --> generate_search_assets
    search --> process_assistants_api_respo

And render the Mermaid code ..

![Mermaid Representation](../images/search_func_tree.png)

In [ ]:
ss = "create_doc_chunks_with_doc_int_markdown,create_image_doc_chunks,create_pdf_chunks,create_table_doc_chunks_markdown,delete_pdf_chunks,extract_doc_using_doc_int,extract_docx_using_py_docx,extract_tables_from_images,extract_xlsx_using_openpyxl,generate_analysis_for_text,generate_document_wide_summary,generate_document_wide_tags,generate_tags_for_all_chunks,pdf_extract_high_res_chunk_images,pdf_extract_images,pdf_extract_text,post_process_images,post_process_tables"


for s in ss.split(','):

    prompt = """You are a code analyzer assistant, and you help with generating documentation and code analysis.

    ## START OF CODE
    {code}
    ## END OF CODE

    ## START OF PIPELINES DEFINITIONS
    {{
        ".pdf": {{
            "gpt-4-vision": [
                "create_pdf_chunks", "pdf_extract_high_res_chunk_images", "pdf_extract_text", "pdf_extract_images", "delete_pdf_chunks", "post_process_images", "extract_tables_from_images", "post_process_tables", "generate_tags_for_all_chunks", "generate_document_wide_tags", "generate_document_wide_summary", "generate_analysis_for_text"
            ],
            "document-intelligence": [
                "create_pdf_chunks", "pdf_extract_high_res_chunk_images", "pdf_extract_text", "pdf_extract_images", "delete_pdf_chunks", "extract_doc_using_doc_int", "create_doc_chunks_with_doc_int_markdown", "post_process_images", "generate_tags_for_all_chunks", "generate_document_wide_tags", "generate_document_wide_summary", "generate_analysis_for_text"
            ],
            "hybrid": [
                "create_pdf_chunks", "pdf_extract_high_res_chunk_images", "delete_pdf_chunks", "extract_doc_using_doc_int", "create_doc_chunks_with_doc_int_markdown", "post_process_images", "post_process_tables", "generate_tags_for_all_chunks", "generate_document_wide_tags", "generate_document_wide_summary", "generate_analysis_for_text"
            ]
        }},
        ".docx": {{
            "py-docx": [
                "extract_docx_using_py_docx", "create_doc_chunks_with_doc_int_markdown", "post_process_images", "generate_tags_for_all_chunks", "generate_document_wide_tags", "generate_document_wide_summary", "generate_analysis_for_text"
            ],
            "document-intelligence": [
                "extract_doc_using_doc_int", "create_doc_chunks_with_doc_int_markdown", "post_process_images", "generate_tags_for_all_chunks", "generate_document_wide_tags", "generate_document_wide_summary", "generate_analysis_for_text"
            ]
        }},
        ".xlsx": {{
            "openpyxl": [
                "extract_xlsx_using_openpyxl", "create_table_doc_chunks_markdown", "create_image_doc_chunks", "generate_tags_for_all_chunks", "generate_document_wide_tags", "generate_document_wide_summary", "generate_analysis_for_text"
            ]
        }}
    }}
    ## END OF PIPELINES DEFINITIONS

    In the library above, please generate a concise documentation about the important function '{func_name}'. This Python documentation needs to be concise in around 3 or 4 sentences. The function is part of a processing pipeline that is executed during document ingestion for multimodal information extraction. Some functions are only specific to certain formats as outlined by the Pipelines Definitions above. Do not mention that this function is part of a document ingestion pipeline in your final answer. Comment about the relevance and position of this function in the overall pipeline, and to which formats it applied. Some functions process either text, images or tables, and others process all 3 modalities (or a combination), so please make sure to mention explicitly which modalities this function processes. 

    """

    p = prompt.format(code = read_asset_file("../code/doc_utils.py")[0], func_name = s)

    output = ask_LLM(p, model_info=model_info)
    print(output)
    





Calling OpenAI APIs with 2 messages - Model: gpt-4 - Endpoint: https://temp-aoai-tst-swc.openai.azure.com//openai/

The `create_doc_chunks_with_doc_int_markdown` function is integral to the processing of documents, particularly when utilizing the Document Intelligence service. It's designed to handle the markdown conversion of document chunks, ensuring that the extracted data is formatted correctly for further analysis. This function is applicable to various document formats and is capable of processing text, images, and tables, making it versatile in the multimodal information extraction process. Its role is crucial in structuring the raw extracted data into a more accessible and analyzable form.

Calling OpenAI APIs with 2 messages - Model: gpt-4 - Endpoint: https://temp-aoai-tst-swc.openai.azure.com//openai/

The `create_image_doc_chunks` function is integral to the processing of image data within multimodal documents. It specifically targets the extraction and organization of imag